<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/evaluation_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 12 13:25:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Perplexity

# Diversity

In [ ]:
!pip install nltk

In [ ]:
from collections import Counter

from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np

In [ ]:
def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        words = seq.split()
        unigrams = Counter(words)
        bigrams = Counter(zip(words, words[1:]))
        # print(unigrams)
        # print(bigrams)

        intra_dist1.append((len(unigrams)+1e-12) / (len(words)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(words)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    # print("\n")
    # print("unigrams all: ", unigrams_all)
    # print("bigrams all: ", bigrams_all)
    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)
    # print("\n")
    # print("intra for each sentence (unigram):", intra_dist1)
    # print("intra for each sentence (bigram):", intra_dist2)
    # print("\n")
    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

# cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
# values = distinct(cand)
# print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")


In [ ]:
cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
values = distinct(cand)
print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")

In [ ]:
def get_list_of_sentences(file_name, label = "    labels:"):
    with open(file_name) as file:
        lines = file.readlines()
    i =0
    dialog_score_pairs = []
    turns = []
    list_diversity_scores = []
    for line in lines:
        if ("NEW EPISODE" in line):
            if len(turns) > 0 : 
                diversity_scores = distinct(turns)
                list_diversity_scores.append(diversity_scores)
                dialog_score_pairs.append([turns, diversity_scores])

            turns = []
        if (line.startswith(label)):
            turns.append(line[len(label)+1:len(line)-3])
    
    average_diversity_score = [
        np.average([item[0] for item in list_diversity_scores]),
        np.average([item[1] for item in list_diversity_scores]),
        np.average([item[2] for item in list_diversity_scores]),
        np.average([item[3] for item in list_diversity_scores])
    ]

    average_of_turns = np.average([len(item[0]) for item in dialog_score_pairs])
    
    return [dialog_score_pairs, average_of_turns, average_diversity_score]

# file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"
# labels = get_list_of_sentences(file_name4, "    labels:")
# values4 = distinct(labels)
# labels

In [ ]:
file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"

labels_n_scores = get_list_of_sentences(file_name4)
print("labels:")
print(f"turns average count: {round(labels_n_scores[1],2)}\n",
    f"intra_distinc1: {labels_n_scores[2][0]}\n", 
    f"intra_distinc2: {labels_n_scores[2][1]}\n", 
    f"inter_distinc1: {labels_n_scores[2][2]}\n", 
    f"inter_distinc2: {labels_n_scores[2][3]}\n")


generated_sentences4_n_scores = get_list_of_sentences(file_name4, "     model:")
# print(f"number of generated sentences in 4th model: {len(generated_sentences4)}")

print("4:")
print(f"turns average count: {round(generated_sentences4_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences4_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences4_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences4_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences4_n_scores[2][3]}\n")

file_name5 = "models-examples\\05-finetuned-beam_prmts_infrnc-testset-output.txt"
generated_sentences5_n_scores = get_list_of_sentences(file_name5, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("5:")
print(f"turns average count: {round(generated_sentences5_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences5_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences5_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences5_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences5_n_scores[2][3]}\n")

file_name9 = "models-examples\\09-finetuned-400m-topk-5epochs-testset-output.txt"
generated_sentences9_n_scores = get_list_of_sentences(file_name9, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("9:")
print(f"turns average count: {round(generated_sentences9_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences9_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences9_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences9_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences9_n_scores[2][3]}\n")

# Coherency

In [3]:
!git lfs install
!git clone https://huggingface.co/Sahajtomar/french_semantic

# !pip uninstall sentence-transformers
!pip install sentence-transformers

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
fatal: destination path 'french_semantic' already exists and is not an empty directory.


In [4]:
output_lines = []
with open("/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt") as f:
    output_lines = f.readlines()
print(len(output_lines))
    
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Sahajtomar/french_semantic')

12121


In [6]:
# calculating the baseline
text, label, generated, context = "", "", "", ""
turns = []

list_of_similarity_scores_for_turns_in_a_dialog = []
list_of_similarity_scores_for_every_dialogs = []

for i,line in enumerate(output_lines):
    split_line = line.split('\t')
    text, label, episode_done = split_line[0].replace("text:", "").replace("\n",""), \
                                split_line[1].replace("labels:","").replace("\n",""), \
                                True if len(split_line) == 3 and split_line[2] else False
    # print("%r"%text, "%r"%label, episode_done)
    label = ' '.join(label.split(' ')[-350:])
    embeddings_label = model.encode(label, convert_to_tensor=True)

    turns.append(text)
    context = ' '.join(turns)
    context = ' '.join(context.split(' ')[-350:]) # if the context is so long, truncate it and pick up the latest 350 token
    embeddings_context = model.encode(context, convert_to_tensor=True)

    cosine_similarity = util.pytorch_cos_sim(embeddings_label, embeddings_context)
    list_of_similarity_scores_for_turns_in_a_dialog.append(cosine_similarity.item())

    # print(i, "\n", "text:", text)
    # print("label:", label)
    # print("\tcontext:",context, "\n")

    turns.append(label)
        
    if (episode_done):
        average_similarity_score = sum(list_of_similarity_scores_for_turns_in_a_dialog) / len(list_of_similarity_scores_for_turns_in_a_dialog)
        list_of_similarity_scores_for_every_dialogs.append(average_similarity_score)
        context = ""
        label = ""
        turns = []

        if len(list_of_similarity_scores_for_every_dialogs) % 50 == 0:
            print(f"{i}: Similarity Score for dialog No. {len(list_of_similarity_scores_for_every_dialogs)}:", average_similarity_score)
        # break
    
    # if len(list_of_similarity_scores_for_every_dialogs) > 5: break


print("Count: ", len(list_of_similarity_scores_for_every_dialogs))
similarity_score_baseline = sum(list_of_similarity_scores_for_every_dialogs)/ len(list_of_similarity_scores_for_every_dialogs)
print("Average: ", similarity_score_baseline)
print(list_of_similarity_scores_for_every_dialogs)

305: Similarity Score for dialog No. 50: 0.33320259605382196
597: Similarity Score for dialog No. 100: 0.34341821501523656
901: Similarity Score for dialog No. 150: 0.33614284581527476
1186: Similarity Score for dialog No. 200: 0.3389923092866004
1487: Similarity Score for dialog No. 250: 0.33712371453888196
1812: Similarity Score for dialog No. 300: 0.3365132758518197
2128: Similarity Score for dialog No. 350: 0.33844597904057977
2417: Similarity Score for dialog No. 400: 0.3395773991626573
2745: Similarity Score for dialog No. 450: 0.34303122636972666
3036: Similarity Score for dialog No. 500: 0.3440167901226082
3358: Similarity Score for dialog No. 550: 0.3422341349603451
3673: Similarity Score for dialog No. 600: 0.34186818995147994
3995: Similarity Score for dialog No. 650: 0.34100669432252934
4321: Similarity Score for dialog No. 700: 0.3402522015377788
4639: Similarity Score for dialog No. 750: 0.3396448607716091
4949: Similarity Score for dialog No. 800: 0.339023196073249
5277: